In [5]:
#IMPORT LIBRARIES

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
!pip -q install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip -q install folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!pip install beautifulsoup4
# Import Beautiful SOup Library for HTML Page Processing
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


In [7]:
# DOwnoad HTML Wikipedia Page for Toronto POstal Codes 
!wget -q -O 'postalCodesCanada.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood  
2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.  
3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.  
4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.  
5. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.  

##### Extract Table content from Wikipedia URL for Neighborhood Data

In [177]:
# Get Data from Wikipedia
wikiToronto = 'postalCodesCanada.html' 

with open(wikiToronto) as fp:
    soup = BeautifulSoup(fp)

table = soup.find_all("table", class_ = "wikitable sortable")

ds = []
j= 0
for tr in table[0].find_all('tr'):
    R = []
    i = 0
    for td in tr.find_all('td'):
        i = i+1
        if str(td.contents[0]).find("href") > 0:
            text = td.find_all('a')[0].contents
        else:
            text = td.contents
        R.append(text[0])
    ds.append(R)
    j = j+1

TOR = pd.DataFrame(ds)

#Remove Top Buffer Row
TOR.drop(TOR.index[0],inplace = True)

#Print Shape and DataSet
print(TOR.shape)
TOR.head()

(287, 3)


,0,1,2
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


##### Modify the Data Frame as per Requirement

In [187]:
#1 The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
columns_ds_Toronto = ['PostalCode','Borough','Neighborhood']
TOR.columns = columns_ds_Toronto

#2 Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
TOR.drop(TOR[TOR['Borough'] == 'Not assigned'].index,inplace = True)
TOR.reset_index(drop=True,inplace = True)

#4 If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
TOR['Neighborhood'].loc[(TOR['Neighborhood']=='Not assigned')] = TOR['Borough'].loc[(TOR['Neighborhood']=='Not assigned')]

#3 Combine neighborhood with same Postal Code. 
neigh_combined = pd.DataFrame(TOR.groupby('PostalCode')['Neighborhood'].apply(', '.join).reset_index())

TOR = TOR.merge(neigh_combined,left_on = 'PostalCode',right_on = 'PostalCode')
#Delete Old Neighborhood Column
del TOR['Neighborhood_x']
TOR.columns = columns_ds_Toronto

# Remove Duplicates
TOR = TOR.drop_duplicates()
print(len(TOR['PostalCode'].unique()))

#Clean Column Data
TOR['Neighborhood'] = TOR['Neighborhood'].str.replace('\n','')

TOR.head()

103


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor, Lawrence Hei..."
4,M7A,Downtown Toronto,Queen's Park


In [186]:
# Final Dataframe Shape
print('Shape : ',TOR.shape)

Shape :  (103, 3)
